In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


In [10]:
grid_data_file = os.path.join(os.getcwd().split('data')[0], 'bi.pickle')
with open(grid_data_file,'rb') as f:
    load_array= pickle.load(f)
exc = [load_array[0][20], load_array[0][50]]
inh = [load_array[1][20], load_array[1][50]]

folder = str(os.getcwd().split('data_2')[1])
print(folder)
case = str(folder[1])

file_ = 'down.pickle'

c_var = [0,1]
if len(folder) == 4:
    if folder[3] == 'e':
        p_var = [0]
    elif folder[3] == 'i':
        p_var = [1]
else:
    p_var = [0,1]
    
print(p_var)

step1 = 100
step2 = 10
step3 = 1

/1_ei
[0, 1]


In [3]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [4]:
d_array = np.arange(500., 20, -0.1)
t0_array = d_array - 20.
t_pen = np.zeros(( t0_array.shape ))
for i in range(len(t_pen)):
    t_pen[i] = t0_array[i] / d_array[i]

In [23]:
dur_pre = 10
dur_post = 10

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

wp = 1. / 20.

tol = 1e-8
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

if case in ['1', '2']:    # low to high
    max_I = [3., -3.]
else:
    max_I = [-3., 3.]
    
if case in ['1', '3']:    # sparsity
    factor_ws = 1.
    factor_we = 0.
else:
    factor_ws = 0.
    factor_we = 1.
    
maxC = [1000., -1000., 0.18, 0.]
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

max_perc_ind = 1

In [15]:
bestControl_ = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_ = np.array( [[None] * len(t_pen)] * len(exc) )
cost_ = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_ = np.array( [[None] * len(t_pen)] * len(exc) )
weights_ = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_ = np.array( [[None] * len(t_pen)] * len(exc) )

initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

print(bestControl_.shape)

(2, 4800)


In [20]:
if os.path.isfile(file_) :
    print("file found")
    
    with open(file_,'rb') as f:
        load_array = pickle.load(f)

    bestControl_ = load_array[0]
    bestState_ = load_array[1]
    cost_ = load_array[2]
    costnode_ = load_array[3]
    weights_ = load_array[4]

file found


In [24]:
# get initial parameters and target states

i_range = [0]
data.set_parameters(aln)
max_percent = []

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = 3000.
    
    control0 = aln.getZeroControl()
    control0 = functions.step_control(aln, maxI_ = max_I[0])

    aln.run(control=control0)
    
    target_rates = np.zeros((2))
    target_rates[0] = aln.rates_exc[0,-1] 
    target_rates[1] = aln.rates_inh[0,-1]

    control0 = functions.step_control(aln, maxI_ = max_I[1])
    aln.run(control=control0)

    init_state_vars = np.zeros(( len(state_vars) ))
    for j in range(len(state_vars)):
        if aln.state[state_vars[j]].size == 1:
            init_state_vars[j] = aln.state[state_vars[j]][0]
        else:
            init_state_vars[j] = aln.state[state_vars[j]][0,-1]

    initVars[i] = init_state_vars
    target[i] = [target_rates[0], target_rates[1]]
    
    max_percent.append( (np.argmax(np.abs(bestControl_[i][0]), axis=2)[0,max_perc_ind] -100) / 4800 )

-------  0 0.4500000000000001 0.4750000000000002


In [25]:
# get uncontrolled cost

data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    dur = d_array[0]
    aln.params.duration = dur
    
    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]
            
    cost.setParams(wp, 0.0, 0.0)

    setinit(initVars[i], aln)
    control0 = aln.getZeroControl()

    # "HS", "FR", "PR", "HZ"
    cgv = None
    max_it = 0

    bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])
    
    cost_uncontrolled[i] = cost_init_[0]

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  0.05 0.0 0.0
interpolate adjoint :  True True True
initial cost =  2951.876147655744
RUN  0 , total integrated cost =  2951.876147655744
Gradient descend method:  None
RUN  0 , total integrated cost =  2951.876147655744
Improved over  0  iterations in  0.0  seconds by  0.0  percent.


In [36]:
for j in np.arange(4600,4800,1):

    #bestControl_[1][j] = None
    convergence_[0][j] = None

In [37]:
tol = 1e-8

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0
    step = step1

    while j < 4799:

        if j == 0:
            j += step
            continue
            
        if not convergence_[i][j-step]:
            if j - step > 0:
                j -= step
                print('go back to -----', j)

        print('-----', j)

        if convergence_[i][j]:
            j += step
            continue

        dur = d_array[j]
        T = 5001 - j
        aln.params.duration = dur

        cost.setParams(wp, 0., 1.)
        weights_[i][j] = cost.getParams()

        setinit(initVars[i], aln)

        max_it = 1000

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]
        
        max_perc_ind = 1
        if i == 1:
            max_perc_ind = 0

        if type(bestControl_[i][j]) == type(None):
            max_percent_prev = (np.argmax(np.abs(bestControl_[i][j-step]), axis=2)[0,max_perc_ind] - 100) / (T-200)
            shift_ind = 0
            if j < 4600:
                while max_percent_prev > max_percent[i]:
                    shift_ind += 1
                    max_percent_prev = (np.argmax(np.abs(bestControl_[i][j-step]), axis=2)[0,max_perc_ind] - 100 - shift_ind) / (T-200)

            control0 = np.zeros(( bestControl_[i][j-step][:,:,100:-(100+step)].shape ))
            print('shift ind = ', shift_ind)
            if shift_ind != 0:
                control0[:,:,:-shift_ind] = 1. * bestControl_[i][j-step][:,:,100+shift_ind:-(100+step)]
            else:
                control0[:,:,:] = 1. * bestControl_[i][j-step][:,:,100:-(100+step)]
                #control0[0,0,:] = 0.
        else:
            if bestControl_[i][j].shape[2] == T+200:
                control0 = bestControl_[i][j][:,:,100:-100]
                #control0[0,1,:] = 0.
            else:
                control0 = bestControl_[i][j][:,:,100:-(100+j)]

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])
        
        time_ = np.arange(-dur_pre,round(dur+dur_post+aln.params.dt,1),aln.params.dt)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        print(costnode_[i][j][0][0][:], costnode_[i][j][2][0][:2])
        

        if cost_[i][j][-1] == 0.:
            convergence_[i][j] = True
            if j >= 4000:
                step = step2
            if j >=4790:
                step = step3

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

        j += step

-------  0 0.4500000000000001 0.4750000000000002
----- 100
----- 200
----- 300
----- 400
----- 500
----- 600
----- 700
----- 800
----- 900
----- 1000
----- 1100
----- 1200
----- 1300
----- 1400
----- 1500
----- 1600
----- 1700
----- 1800
----- 1900
----- 2000
----- 2100
----- 2200
----- 2300
----- 2400
----- 2500
----- 2600
----- 2700
----- 2800
----- 2900
----- 3000
----- 3100
----- 3200
----- 3300
----- 3400
----- 3500
----- 3600
----- 3700
----- 3800
----- 3900
----- 4000
----- 4100
----- 4200
----- 4300
----- 4400
----- 4500
----- 4600
set cost params:  0.05 0.0 1.0
interpolate adjoint :  True True True
initial cost =  5.137321849056868
RUN  0 , total integrated cost =  5.137321849056868
Gradient descend method:  None
RUN  1 , total integrated cost =  5.137321849056868
Control only changes marginally.
RUN  1 , total integrated cost =  5.137321849056868
Improved over  1  iterations in  0.07548525498714298  seconds by  0.0  percent.
[0.11073908 0.05250223] [0.72987919 4.39928059]
---

----- 4770
set cost params:  0.05 0.0 1.0
interpolate adjoint :  True True True
initial cost =  16.20860142113916
RUN  0 , total integrated cost =  16.20860142113916
Gradient descend method:  None
RUN  1 , total integrated cost =  16.20860142113916
Control only changes marginally.
RUN  1 , total integrated cost =  16.20860142113916
Improved over  1  iterations in  0.05534250900382176  seconds by  0.0  percent.
[0.84512471 0.35391365] [11.19054974  4.95809976]
----- 4780
set cost params:  0.05 0.0 1.0
interpolate adjoint :  True True True
initial cost =  18.484119920459513
RUN  0 , total integrated cost =  18.484119920459513
Gradient descend method:  None
RUN  1 , total integrated cost =  18.484119920459513
Control only changes marginally.
RUN  1 , total integrated cost =  18.484119920459513
Improved over  1  iterations in  0.08571805999963544  seconds by  0.0  percent.
[2.04544184 0.34877705] [13.25910471  5.10530427]
----- 4790
set cost params:  0.05 0.0 1.0
interpolate adjoint :  Tru

In [194]:
bestControl_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
cost_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
weights_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_maxW = np.array( [[None] * len(t_pen)] * len(exc) )

file_maxW = 'down_Wmax.pickle'

In [205]:
if os.path.isfile(file_maxW) :
    print("file found")
    
    with open(file_maxW,'rb') as f:
        load_array = pickle.load(f)

    bestControl_maxW = load_array[0]
    bestState_maxW = load_array[1]
    cost_maxW = load_array[2]
    costnode_maxW = load_array[3]
    weights_maxW = load_array[4]

file found


In [200]:
for i in i_range:
    print('------------', i)
    j = 0

    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.

    aln.params.duration = dur

    while not convergence_maxW[i][j]:
        dur = d_array[j]
        aln.params.duration = dur

        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        if type(bestControl_maxW[i][j]) == type(None):
            max_it = 10
            control0 = bestControl_[i][0][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j]) 
                       - sum(costnode_[i][j][0][0][:]) ) / sum( costnode_[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                       - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            max_it = int(100)

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_maxW[i][j][0][0][0], costnode_maxW[i][j][2][0][:2])

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

------------ 0
set cost params:  1.0 0.0 440952.9295571259
interpolate adjoint :  True True True
initial cost =  1468888.8173558703
RUN  0 , total integrated cost =  1468888.8173558703
Gradient descend method:  None
RUN  1 , total integrated cost =  1468888.8173558703
Control only changes marginally.
RUN  1 , total integrated cost =  1468888.8173558703
Improved over  1  iterations in  1.243575089960359  seconds by  0.0  percent.
cost splitting :  0.0004690330126152677 [3.88250122e-18 3.33116920e+00]
------------ 1
set cost params:  1.0 0.0 294679.3073488042
interpolate adjoint :  True True True
initial cost =  1475251.0595766376
RUN  0 , total integrated cost =  1475251.0595766376
Gradient descend method:  None
RUN  1 , total integrated cost =  1475251.0595766376
Control only changes marginally.
RUN  1 , total integrated cost =  1475251.0595766376
Improved over  1  iterations in  1.3007536130025983  seconds by  0.0  percent.
cost splitting :  0.00013038254234949543 [5.00629336e+00 2.23

In [237]:
for j in np.arange(100,4800,1):
    #bestControl_maxW[1][j] = None
    convergence_maxW[1][j] = None

In [238]:
tol = 1e-7

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0
    step = step1
    
    if i == 0:
        continue

    while j < 4800:
        
        #if j == 4000:
        #    break

        if j == 0:
            j += step
            continue
            
        if not convergence_maxW[i][j-step]:
            if j - step > 0:
                j -= step
                print('go back to -----', j)

        print('-----', j)

        if convergence_maxW[i][j]:
            j += step
            continue

        dur = d_array[j]
        T = 5001 - j
        aln.params.duration = dur
        setinit(initVars[i], aln)

        max_it = 1000

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]
        
        shiftmax_ind = 0
        if i == 0:
            shiftmax_ind = 1

        if type(bestControl_maxW[i][j]) == type(None):
            max_percent_prev = (np.argmax(np.abs(bestControl_maxW[i][j-step]), axis=2)[0,shiftmax_ind] - 100) / (T-200)
            shift_ind = 0
            if j < 4000:
                while max_percent_prev > max_percent[i]:
                    shift_ind += 1
                    max_percent_prev = (np.argmax(np.abs(bestControl_maxW[i][j-step]), axis=2)[0,shiftmax_ind] - 100 - shift_ind) / (T-200)

            control0 = np.zeros(( bestControl_maxW[i][j-step][:,:,100:-(100+step)].shape ))
            print('shift ind = ', shift_ind)
            if shift_ind != 0:
                control0[:,:,:-shift_ind] = 1.01 * bestControl_maxW[i][j-step][:,:,100+shift_ind:-(100+step)]
            else:
                control0[:,:,:] = 1. * bestControl_maxW[i][j-step][:,:,100:-(100+step)]
                
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                          - sum(costnode_maxW[i][j-step][0][0][:]) ) / sum( costnode_maxW[i][j-step][2][0][:] )
        
        else:
            if bestControl_maxW[i][j].shape[2] == T+200:
                control0 = bestControl_maxW[i][j][:,:,100:-100]
                #control0[0,1,:] = 0.
            else:
                control0 = bestControl_maxW[i][j][:,:,100:-(100+j)]
                
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                         - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
        
        #if j >= 3200:
        #    weight_ *= 0.9
        
        cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
        weights_maxW[i][j] = cost.getParams()

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        print(costnode_maxW[i][j][0][0][:2], costnode_maxW[i][j][2][0][:2])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True
            if j >= 4400:
                step = step2
            if j >=4790:
                step = step3

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

        j += step

-------  0 0.4500000000000001 0.4750000000000002
-------  1 0.47500000000000014 0.6000000000000003
----- 100
set cost params:  1.0 0.0 288785.60457409744
interpolate adjoint :  True True True
initial cost =  1445746.0380187386
RUN  0 , total integrated cost =  1445746.0380187386
Gradient descend method:  None
RUN  1 , total integrated cost =  1445745.4690925165
RUN  2 , total integrated cost =  1445745.4690925165
Control only changes marginally.
RUN  2 , total integrated cost =  1445745.4690925165
Improved over  2  iterations in  2.6791709719691426  seconds by  3.9351740014126335e-05  percent.
[9.06964957e-04 2.82410885e-05] [5.00629340e+00 2.25539562e-09]
go back to ----- 100
----- 100
set cost params:  1.0 0.0 288785.7182163026
interpolate adjoint :  True True True
initial cost =  1445746.0380187389
RUN  0 , total integrated cost =  1445746.0380187389
Gradient descend method:  None
RUN  1 , total integrated cost =  1445746.0380187389
Control only changes marginally.
RUN  1 , total in

RUN  1 , total integrated cost =  1091685.7838099846
Control only changes marginally.
RUN  1 , total integrated cost =  1091685.7838099846
Improved over  1  iterations in  0.9665586249902844  seconds by  0.0  percent.
[0.07997905 0.03157515] [5.00629569e+00 2.54143465e-09]
----- 1400
set cost params:  1.0 0.0 212168.76598557486
interpolate adjoint :  True True True
initial cost =  1062180.7626259224
RUN  0 , total integrated cost =  1062180.7626259224
Gradient descend method:  None
RUN  1 , total integrated cost =  1062179.987504835
RUN  2 , total integrated cost =  1062179.987504835
Control only changes marginally.
RUN  2 , total integrated cost =  1062179.987504835
Improved over  2  iterations in  2.001843634992838  seconds by  7.297449874954509e-05  percent.
[0.08756438 0.03457477] [5.00629704e+00 2.56684897e-09]
go back to ----- 1400
----- 1400
set cost params:  1.0 0.0 212168.92081479914
interpolate adjoint :  True True True
initial cost =  1062180.7626259224
RUN  0 , total integr

RUN  1 , total integrated cost =  796635.5719693583
Control only changes marginally.
RUN  1 , total integrated cost =  796635.5719693583
Improved over  1  iterations in  0.6733369919820689  seconds by  0.0  percent.
[2.6192849  1.03804392] [5.00642307e+00 2.80731912e-09]
----- 2400
set cost params:  1.0 0.0 153226.2316587437
interpolate adjoint :  True True True
initial cost =  767130.5507852951
RUN  0 , total integrated cost =  767130.5507852951
Gradient descend method:  None
RUN  1 , total integrated cost =  767130.5507852951
Control only changes marginally.
RUN  1 , total integrated cost =  767130.5507852951
Improved over  1  iterations in  0.6754878450301476  seconds by  0.0  percent.
[3.62650897 1.43881038] [5.00648927e+00 2.83538865e-09]
----- 2500
set cost params:  1.0 0.0 147329.61946355534
interpolate adjoint :  True True True
initial cost =  737625.5296012326
RUN  0 , total integrated cost =  737625.5296012326
Gradient descend method:  None
RUN  1 , total integrated cost =  7

RUN  1 , total integrated cost =  590100.4229377558
RUN  2 , total integrated cost =  590100.1896087598
RUN  3 , total integrated cost =  590099.9252733224
RUN  4 , total integrated cost =  590099.924569115
RUN  5 , total integrated cost =  590099.9200435187
RUN  6 , total integrated cost =  590099.9083535634
RUN  7 , total integrated cost =  590099.9071082157
RUN  8 , total integrated cost =  590099.90547488
RUN  9 , total integrated cost =  590099.8962723911
RUN  10 , total integrated cost =  590099.8942319733
RUN  11 , total integrated cost =  590099.893447829
RUN  12 , total integrated cost =  590099.8672554052
RUN  13 , total integrated cost =  590099.8495513087
RUN  14 , total integrated cost =  590099.8489629674
RUN  15 , total integrated cost =  590099.8318335043
RUN  16 , total integrated cost =  590099.8024886997
RUN  17 , total integrated cost =  590099.8014759775
RUN  18 , total integrated cost =  590099.7997902998
RUN  19 , total integrated cost =  590099.7909181899
RUN  2

RUN  8 , total integrated cost =  442574.8108560292
RUN  9 , total integrated cost =  442574.39320925955
RUN  10 , total integrated cost =  442574.39103195583
RUN  11 , total integrated cost =  442574.3864428331
RUN  12 , total integrated cost =  442574.3674901532
RUN  13 , total integrated cost =  442574.36376249284
RUN  14 , total integrated cost =  442574.36181145994
RUN  15 , total integrated cost =  442574.33362520096
RUN  16 , total integrated cost =  442574.3238915093
RUN  17 , total integrated cost =  442574.32226979604
RUN  18 , total integrated cost =  442574.2564795709
RUN  19 , total integrated cost =  442574.2121619755
RUN  20 , total integrated cost =  442574.21090790175
RUN  30 , total integrated cost =  442573.05422791303
RUN  40 , total integrated cost =  442551.9283179479
RUN  50 , total integrated cost =  442551.45195832825
RUN  60 , total integrated cost =  442550.71591095976
RUN  70 , total integrated cost =  442546.9441684878
RUN  80 , total integrated cost =  442

...

RUN  1 , total integrated cost =  162277.61651190673
RUN  2 , total integrated cost =  162277.61651190673
Control only changes marginally.
RUN  2 , total integrated cost =  162277.61651190673
Improved over  2  iterations in  0.45977658103220165  seconds by  6.335199032037053e-11  percent.
[1194.11188518 1238.82969258] [5.73988676e+00 3.21687460e-09]
go back to ----- 4450
----- 4450
set cost params:  1.0 0.0 27848.053729414794
interpolate adjoint :  True True True
initial cost =  162277.61651200952
RUN  0 , total integrated cost =  162277.61651200952
Gradient descend method:  None
RUN  1 , total integrated cost =  162277.61651200952
Control only changes marginally.
RUN  1 , total integrated cost =  162277.61651200952
Improved over  1  iterations in  0.23051608604146168  seconds by  0.0  percent.
[1194.11188518 1238.82969258] [5.73988676e+00 3.21687460e-09]
----- 4460
set cost params:  1.0 0.0 27151.41224149662
interpolate adjoint :  True True True
initial cost =  159327.1143936032
RUN  

RUN  4 , total integrated cost =  141624.10168030512
Control only changes marginally.
RUN  4 , total integrated cost =  141624.10168030512
Improved over  4  iterations in  0.753612330998294  seconds by  2.0197603589622304e-09  percent.
[1424.56811419 1744.58138117] [6.03596887e+00 2.92352574e-09]
go back to ----- 4520
----- 4520
set cost params:  1.0 0.0 22938.314470756544
interpolate adjoint :  True True True
initial cost =  141624.1016831656
RUN  0 , total integrated cost =  141624.1016831656
Gradient descend method:  None
RUN  1 , total integrated cost =  141624.1016831656
Control only changes marginally.
RUN  1 , total integrated cost =  141624.1016831656
Improved over  1  iterations in  0.20315673801815137  seconds by  0.0  percent.
[1424.56811419 1744.58138117] [6.03596887e+00 2.92352574e-09]
----- 4530
set cost params:  1.0 0.0 22231.508357641105
interpolate adjoint :  True True True
initial cost =  138673.59956475935
RUN  0 , total integrated cost =  138673.59956475935
Gradient

RUN  1 , total integrated cost =  106218.07626229037
Control only changes marginally.
RUN  1 , total integrated cost =  106218.07626229037
Improved over  1  iterations in  0.18886767799267545  seconds by  0.0  percent.
[1519.42585991 3106.32891331] [7.09186946e+00 1.79370467e-09]
----- 4650
set cost params:  1.0 0.0 13571.698515069582
interpolate adjoint :  True True True
initial cost =  103267.57414388408
RUN  0 , total integrated cost =  103267.57414388408
Gradient descend method:  None
RUN  1 , total integrated cost =  103267.57414388408
Control only changes marginally.
RUN  1 , total integrated cost =  103267.57414388408
Improved over  1  iterations in  0.18568916601361707  seconds by  0.0  percent.
[1506.06249138 3327.03197091] [7.25292266e+00 1.70825204e-09]
----- 4660
set cost params:  1.0 0.0 12806.325071102623
interpolate adjoint :  True True True
initial cost =  100317.07202547781
RUN  0 , total integrated cost =  100317.07202547781
Gradient descend method:  None
RUN  1 , tot

RUN  1 , total integrated cost =  64911.04660245403
RUN  2 , total integrated cost =  64911.04660245403
Control only changes marginally.
RUN  2 , total integrated cost =  64911.04660245403
Improved over  2  iterations in  0.31028965197037905  seconds by  3.3099212259912747e-09  percent.
[ 4577.25771464 11571.55779351] [9.46536582 1.46870864]
go back to ----- 4780
----- 4780
set cost params:  1.0 0.0 4459.657860009754
interpolate adjoint :  True True True
initial cost =  64911.04660460253
RUN  0 , total integrated cost =  64911.04660460253
Gradient descend method:  None
RUN  1 , total integrated cost =  64911.04660460253
Control only changes marginally.
RUN  1 , total integrated cost =  64911.04660460253
Improved over  1  iterations in  0.14717904600547627  seconds by  0.0  percent.
[ 4577.25771464 11571.55779351] [9.46536582 1.46870864]
----- 4790
set cost params:  1.0 0.0 4020.9554580124827
interpolate adjoint :  True True True
initial cost =  61960.544486196275
RUN  0 , total integra

RUN  1 , total integrated cost =  59305.092577161486
RUN  2 , total integrated cost =  59305.092577161486
Control only changes marginally.
RUN  2 , total integrated cost =  59305.092577161486
Improved over  2  iterations in  0.2644318880047649  seconds by  4.163496214459883e-09  percent.
[5529.23003074 9644.92887418] [9.6859834  2.40210554]


In [213]:
print(max_percent[1])

0.528125
